In [1]:
%%time
import pandas as pd
it = pd.read_csv("/content/drive/MyDrive/cpe/goodreads_interactions.csv")

CPU times: user 1min 3s, sys: 49.3 s, total: 1min 53s
Wall time: 2min 10s


In [2]:
master = it[it["is_read"] == 1][["user_id", "book_id", "rating"]]
it = 0

In [3]:
from sklearn.model_selection import train_test_split
import numpy as np
train, test = train_test_split(np.arange(master.shape[0]), test_size=0.2, random_state=42, shuffle = True)

In [4]:
train,val = train_test_split(train, test_size=0.2, random_state=42, shuffle = True)

In [5]:
!pip install git+https://github.com/gbolmier/funk-svd
from funk_svd import SVD
svd = SVD(learning_rate=0.001, regularization=0.005, n_epochs=100, n_factors=15, min_rating=0, max_rating=5)

  Cloning https://github.com/gbolmier/funk-svd to /tmp/pip-req-build-unczsc8p
  Running command git clone -q https://github.com/gbolmier/funk-svd /tmp/pip-req-build-unczsc8p
  Created wheel for funk-svd: filename=funk_svd-0.0.1.dev1-cp36-none-any.whl size=8598 sha256=b6748302b9ad834cf98812cb3b988008b0843112b99335fc5c190d16c8b281b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-nyvv3k4i/wheels/66/f1/cb/e8147525b73388cc0bd5588c915e731ab65aba9a968e3ba455
Successfully built funk-svd


In [6]:
master.columns = ["u_id", "i_id", "rating"]

In [7]:
%%time
tr = master.iloc[train]
print("Train Allocated")
vl = master.iloc[val]
print("Val Allocated")
tt = master.iloc[test]
print("Test Allocated")

Train Allocated
Val Allocated
Test Allocated
CPU times: user 23.6 s, sys: 1.16 s, total: 24.8 s
Wall time: 24.6 s


In [8]:
svd.fit(X=tr, X_val=vl, early_stopping=True, shuffle=False)

Preprocessing data...

Epoch 1/100  | val_loss: 1.68 - val_rmse: 1.30 - val_mae: 0.97 - took 23.7 sec
Epoch 2/100  | val_loss: 1.61 - val_rmse: 1.27 - val_mae: 0.95 - took 22.6 sec
Epoch 3/100  | val_loss: 1.57 - val_rmse: 1.25 - val_mae: 0.93 - took 22.8 sec
Epoch 4/100  | val_loss: 1.55 - val_rmse: 1.24 - val_mae: 0.92 - took 22.3 sec
Epoch 5/100  | val_loss: 1.53 - val_rmse: 1.24 - val_mae: 0.92 - took 22.2 sec
Epoch 6/100  | val_loss: 1.52 - val_rmse: 1.23 - val_mae: 0.91 - took 22.8 sec
Epoch 7/100  | val_loss: 1.51 - val_rmse: 1.23 - val_mae: 0.91 - took 23.1 sec
Epoch 8/100  | val_loss: 1.50 - val_rmse: 1.22 - val_mae: 0.91 - took 28.0 sec
Epoch 9/100  | val_loss: 1.49 - val_rmse: 1.22 - val_mae: 0.90 - took 26.8 sec
Epoch 10/100 | val_loss: 1.49 - val_rmse: 1.22 - val_mae: 0.90 - took 28.6 sec
Epoch 11/100 | val_loss: 1.48 - val_rmse: 1.22 - val_mae: 0.90 - took 25.7 sec
Epoch 12/100 | val_loss: 1.48 - val_rmse: 1.22 - val_mae: 0.90 - took 23.4 sec
Epoch 13/100 | val_loss: 1.47

In [9]:
pred = svd.predict(tt)

In [10]:
np.std(pred)

0.6079450626369296

In [11]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(tt['rating'], pred)
print(f'Test MAE: {mae:.2f}')

Test MAE: 0.89


In [12]:
from sklearn.metrics import mean_squared_error
mae = mean_squared_error(tt['rating'], pred)
print(f'Test R2: {mae:.2f}')

Test R2: 1.46
